In [2]:
from os import mkdir

from IPython.display import Image, display
import DataLoader as dl
import pandas as pd


dataloader = dl.DataLoader()

movies_df = dataloader.movies_df
actors_df = dataloader.actors_df
countries_df = dataloader.countries_df
crew_df = dataloader.crew_df
genres_df = dataloader.genres_df
languages_df = dataloader.languages_df
posters_df = dataloader.posters_df
releases_df = dataloader.releases_df
studios_df = dataloader.studios_df
themes_df = dataloader.themes_df

movies_df

,name,date,tagline,description,minute,rating
id,,,,,,
1000001,Barbie,2023.0,She's everything. He's just Ken.,Barbie and Ken are having the time of their li...,114.0,3.86
1000002,Parasite,2019.0,Act like you own the place.,"All unemployed, Ki-taek's family takes peculia...",133.0,4.56
1000003,Everything Everywhere All at Once,2022.0,The universe is so much bigger than you realize.,An aging Chinese immigrant is swept up in an i...,140.0,4.30
1000004,Fight Club,1999.0,Mischief. Mayhem. Soap.,A ticking-time-bomb insomniac and a slippery s...,139.0,4.27
1000005,La La Land,2016.0,Here's to the fools who dream.,"Mia, an aspiring actress, serves lattes to mov...",129.0,4.09
...,...,...,...,...,...,...
1941593,神笛,NaN,NaN,NaN,NaN,NaN
1941594,蟲極道蜜団子抗争編 壱ノ巻,NaN,NaN,Shinjuku forest at night. In the sap taverns o...,30.0,NaN
1941595,蟲極道蜜団子抗争編 弐ノ巻,NaN,NaN,"The city that never sleeps, where insects gath...",30.0,NaN


In [3]:
print("Movie columns:" , movies_df.columns)
print("Actors columns:" , actors_df.columns)
print("Countries columns:" , countries_df.columns)
print("Crew columns:" , crew_df.columns)
print("Genres columns:" , genres_df.columns)
print("Languages columns:" , languages_df.columns)
print("Posters columns:" , posters_df.columns)
print("Release columns:" , releases_df.columns)
print("Studios columns:" , studios_df.columns)
print("Themes columns:" , themes_df.columns)

Movie columns: Index(['name', 'date', 'tagline', 'description', 'minute', 'rating'], dtype='object')
Actors columns: Index(['name', 'role'], dtype='object')
Countries columns: Index(['country'], dtype='object')
Crew columns: Index(['role', 'name'], dtype='object')
Genres columns: Index(['genre'], dtype='object')
Languages columns: Index(['type', 'language'], dtype='object')
Posters columns: Index(['link'], dtype='object')
Release columns: Index(['country', 'date', 'type', 'rating'], dtype='object')
Studios columns: Index(['studio'], dtype='object')
Themes columns: Index(['theme'], dtype='object')


# Data Cleaning
## Removing or replacing all the data that are not compliant with the dataset
### Dataset types:


fetch and cast the data to their correct type

In [4]:
movies_df.dtypes

name            object
date           float64
tagline         object
description     object
minute         float64
rating         float64
dtype: object

In [5]:
movies_df['name'] = movies_df['name'].astype(pd.StringDtype())
movies_df['tagline'] = movies_df['tagline'].astype(pd.StringDtype())
movies_df['description'] = movies_df['description'].astype(pd.StringDtype())
movies_df['date'] = movies_df['date'].astype('Int64')

In [6]:
movies_df.dtypes

name           string[python]
date                    Int64
tagline        string[python]
description    string[python]
minute                float64
rating                float64
dtype: object

In [7]:
actors_df.dtypes

name    object
role    object
dtype: object

In [8]:
actors_df['name'] = actors_df['name'].astype(pd.StringDtype())
actors_df['role'] = actors_df['role'].astype(pd.StringDtype())

In [9]:
countries_df.dtypes

country    object
dtype: object

In [10]:
countries_df['country'] = countries_df['country'].astype(pd.StringDtype())

In [11]:
crew_df.dtypes

role    object
name    object
dtype: object

In [12]:
crew_df['role'] = crew_df['role'].astype(pd.StringDtype())
crew_df['name'] = crew_df['name'].astype(pd.StringDtype())

In [13]:
genres_df.dtypes

genre    object
dtype: object

In [14]:
genres_df['genre'] = genres_df['genre'].astype(pd.StringDtype())

In [15]:
languages_df.dtypes

type        object
language    object
dtype: object

In [16]:
languages_df['type'] = languages_df['type'].astype(pd.StringDtype())
languages_df['language'] = languages_df['language'].astype(pd.StringDtype())

In [17]:
posters_df.dtypes

link    object
dtype: object

In [18]:
posters_df['link'] = posters_df['link'].astype(pd.StringDtype())

In [19]:
releases_df.dtypes

country    object
date       object
type       object
rating     object
dtype: object

In [20]:
releases_df['country'] = releases_df['country'].astype(pd.StringDtype())
releases_df['date'] = releases_df['date'].astype(pd.StringDtype()) #todo: change to date type
releases_df['type'] = releases_df['type'].astype(pd.StringDtype())
releases_df['rating'] = releases_df['rating'].astype(pd.StringDtype())

In [21]:
studios_df.dtypes

studio    object
dtype: object

In [22]:
studios_df['studio'] = studios_df['studio'].astype(pd.StringDtype())

In [23]:
themes_df.dtypes

theme    object
dtype: object

In [24]:
themes_df['theme'] = themes_df['theme'].astype(pd.StringDtype())

fetching the movies without a name

In [25]:
movies_df[movies_df['name'].isna()]

,name,date,tagline,description,minute,rating
id,,,,,,
1287515,<NA>,2015,<NA>,NONE is a short film that explores the balance...,4.0,NaN
1617643,<NA>,<NA>,<NA>,<NA>,NaN,NaN
1646521,<NA>,2008,<NA>,<NA>,NaN,NaN
1648186,<NA>,<NA>,<NA>,<NA>,NaN,NaN
1720295,<NA>,<NA>,<NA>,"In this directorial debut of Eden Ewardson, he...",8.0,NaN
1725370,<NA>,<NA>,<NA>,<NA>,NaN,NaN
1741482,<NA>,<NA>,<NA>,<NA>,90.0,NaN
1840338,<NA>,<NA>,<NA>,<NA>,NaN,NaN
1883229,<NA>,<NA>,<NA>,<NA>,NaN,NaN


### Movies:

by joining the subset of the movie without a name we can ensure that no reference are lost during the process (no other part of the dataset actually contain some data about those movies)

In [26]:
movies_na_subset = pd.DataFrame(movies_df[movies_df['name'].isna()])

#movies_na_subset join actors
movies_na_subset.join(actors_df, on='id', lsuffix='_movies', rsuffix='_actors')

,name_movies,date,tagline,description,minute,rating,name_actors,role
id,,,,,,,,
1287515,<NA>,2015,<NA>,NONE is a short film that explores the balance...,4.0,NaN,<NA>,<NA>
1617643,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
1646521,<NA>,2008,<NA>,<NA>,NaN,NaN,Catherine R,Self
1648186,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
1720295,<NA>,<NA>,<NA>,"In this directorial debut of Eden Ewardson, he...",8.0,NaN,<NA>,<NA>
1725370,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
1741482,<NA>,<NA>,<NA>,<NA>,90.0,NaN,<NA>,<NA>
1840338,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
1883229,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>


In [27]:
#movies_na_subset join countries
movies_na_subset.join(countries_df, on='id', lsuffix='_movies', rsuffix='_countries')

,name,date,tagline,description,minute,rating,country
id,,,,,,,
1287515,<NA>,2015,<NA>,NONE is a short film that explores the balance...,4.0,NaN,<NA>
1617643,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1646521,<NA>,2008,<NA>,<NA>,NaN,NaN,USA
1648186,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1720295,<NA>,<NA>,<NA>,"In this directorial debut of Eden Ewardson, he...",8.0,NaN,<NA>
1725370,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1741482,<NA>,<NA>,<NA>,<NA>,90.0,NaN,<NA>
1840338,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1883229,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>


In [28]:
#movies_na_subset join crew
movies_na_subset.join(crew_df, on='id', lsuffix='_movies', rsuffix='_crew')

,name_movies,date,tagline,description,minute,rating,role,name_crew
id,,,,,,,,
1287515,<NA>,2015,<NA>,NONE is a short film that explores the balance...,4.0,NaN,Director,Ash Thorp
1287515,<NA>,2015,<NA>,NONE is a short film that explores the balance...,4.0,NaN,Composer,Ben Lukas Boysen
1617643,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
1646521,<NA>,2008,<NA>,<NA>,NaN,NaN,Director,Giovanni De Nava
1648186,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
1720295,<NA>,<NA>,<NA>,"In this directorial debut of Eden Ewardson, he...",8.0,NaN,<NA>,<NA>
1725370,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
1741482,<NA>,<NA>,<NA>,<NA>,90.0,NaN,<NA>,<NA>
1840338,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>


In [29]:
#movies_na_subset join genres
movies_na_subset.join(genres_df, on='id', lsuffix='_movies', rsuffix='_genres')

,name,date,tagline,description,minute,rating,genre
id,,,,,,,
1287515,<NA>,2015,<NA>,NONE is a short film that explores the balance...,4.0,NaN,Animation
1617643,<NA>,<NA>,<NA>,<NA>,NaN,NaN,Documentary
1646521,<NA>,2008,<NA>,<NA>,NaN,NaN,Documentary
1648186,<NA>,<NA>,<NA>,<NA>,NaN,NaN,Thriller
1720295,<NA>,<NA>,<NA>,"In this directorial debut of Eden Ewardson, he...",8.0,NaN,<NA>
1725370,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1741482,<NA>,<NA>,<NA>,<NA>,90.0,NaN,<NA>
1840338,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1883229,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>


In [30]:
#movies_na_subset join languages
movies_na_subset.join(languages_df, on='id', lsuffix='_movies', rsuffix='_languages')

,name,date,tagline,description,minute,rating,type,language
id,,,,,,,,
1287515,<NA>,2015,<NA>,NONE is a short film that explores the balance...,4.0,NaN,<NA>,<NA>
1617643,<NA>,<NA>,<NA>,<NA>,NaN,NaN,Language,English
1646521,<NA>,2008,<NA>,<NA>,NaN,NaN,Language,English
1648186,<NA>,<NA>,<NA>,<NA>,NaN,NaN,Language,English
1720295,<NA>,<NA>,<NA>,"In this directorial debut of Eden Ewardson, he...",8.0,NaN,Language,Burmese
1725370,<NA>,<NA>,<NA>,<NA>,NaN,NaN,Language,English
1741482,<NA>,<NA>,<NA>,<NA>,90.0,NaN,Language,English
1840338,<NA>,<NA>,<NA>,<NA>,NaN,NaN,Language,English
1883229,<NA>,<NA>,<NA>,<NA>,NaN,NaN,Language,English


In [31]:
#movies_na_subset join posters
movies_na_subset.join(posters_df, on='id', lsuffix='_movies', rsuffix='_posters')

,name,date,tagline,description,minute,rating,link
id,,,,,,,
1287515,<NA>,2015,<NA>,NONE is a short film that explores the balance...,4.0,NaN,https://a.ltrbxd.com/resized/film-poster/4/4/7...
1617643,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1646521,<NA>,2008,<NA>,<NA>,NaN,NaN,https://a.ltrbxd.com/resized/film-poster/1/0/1...
1648186,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1720295,<NA>,<NA>,<NA>,"In this directorial debut of Eden Ewardson, he...",8.0,NaN,<NA>
1725370,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1741482,<NA>,<NA>,<NA>,<NA>,90.0,NaN,https://a.ltrbxd.com/resized/film-poster/6/5/6...
1840338,<NA>,<NA>,<NA>,<NA>,NaN,NaN,https://a.ltrbxd.com/resized/film-poster/5/8/4...
1883229,<NA>,<NA>,<NA>,<NA>,NaN,NaN,https://a.ltrbxd.com/resized/film-poster/7/4/1...


In [32]:
#movies_na_subset join releases
movies_na_subset.join(releases_df, on='id', lsuffix='_movies', rsuffix='_releases')

,name,date_movies,tagline,description,minute,rating_movies,country,date_releases,type,rating_releases
id,,,,,,,,,,
1287515,<NA>,2015,<NA>,NONE is a short film that explores the balance...,4.0,NaN,USA,2015-01-01,Theatrical,<NA>
1617643,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1646521,<NA>,2008,<NA>,<NA>,NaN,NaN,USA,2008-03-02,Theatrical,PG
1648186,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1720295,<NA>,<NA>,<NA>,"In this directorial debut of Eden Ewardson, he...",8.0,NaN,<NA>,<NA>,<NA>,<NA>
1725370,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1741482,<NA>,<NA>,<NA>,<NA>,90.0,NaN,<NA>,<NA>,<NA>,<NA>
1840338,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1883229,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>


In [33]:
#movies_na_subset join studios
movies_na_subset.join(studios_df, on='id', lsuffix='_movies', rsuffix='_studios')

,name,date,tagline,description,minute,rating,studio
id,,,,,,,
1287515,<NA>,2015,<NA>,NONE is a short film that explores the balance...,4.0,NaN,<NA>
1617643,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1646521,<NA>,2008,<NA>,<NA>,NaN,NaN,<NA>
1648186,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1720295,<NA>,<NA>,<NA>,"In this directorial debut of Eden Ewardson, he...",8.0,NaN,<NA>
1725370,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1741482,<NA>,<NA>,<NA>,<NA>,90.0,NaN,<NA>
1840338,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1883229,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>


In [34]:
#movies_na_subset join themes
movies_na_subset.join(themes_df, on='id', lsuffix='_movies', rsuffix='_themes')

,name,date,tagline,description,minute,rating,theme
id,,,,,,,
1287515,<NA>,2015,<NA>,NONE is a short film that explores the balance...,4.0,NaN,<NA>
1617643,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1646521,<NA>,2008,<NA>,<NA>,NaN,NaN,<NA>
1648186,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1720295,<NA>,<NA>,<NA>,"In this directorial debut of Eden Ewardson, he...",8.0,NaN,<NA>
1725370,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1741482,<NA>,<NA>,<NA>,<NA>,90.0,NaN,<NA>
1840338,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>
1883229,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>


## Looking at the joined table it is possible to make some correction at the dataset

movie with id 1287515: looking at the description and at the poster and after a quick search on internet it is possible to evaluate that the name of the Movie is ***NONE***, no it is possible to update the name in the table

In [35]:
display(Image(url=posters_df['link'][1287515], width=200, height=300))

In [36]:
movies_df.loc[1287515,'name'] = "NONE"

movie with id: 1720295 looking at the description and after a quick search on internet it is possible to evaluate that the name of the Movie is ***Memorandum of Softness Green***

In [37]:
movies_df.loc[1720295, 'name'] = "Memorandum of Softness Green"

movie with id 1741482: looking at the poster it is possible to name the movie: ***Sweethurt*** but this time the record for this movie is a duplicate (see 1254818) so all the reference will be deleted

In [38]:
movies_df.drop(1720295)
actors_df.drop(1720295, errors='ignore')
countries_df.drop(1720295, errors='ignore')
crew_df.drop(1720295, errors='ignore')
genres_df.drop(1720295, errors='ignore')
languages_df.drop(1720295, errors='ignore')
posters_df.drop(1720295, errors='ignore')
releases_df.drop(1720295, errors='ignore')
studios_df.drop(1720295, errors='ignore')
themes_df.drop(1720295, errors='ignore')
print()

movie with id 1840338: looking at the poster it is possible to name the movie: ***The Find*** and the director is ***Daniel Dahlstrom***

In [39]:
display(Image(url=posters_df['link'][1840338], width=200, height=300))

In [40]:
movies_df.loc[1840338,'name'] = "The Find"
crew_df.loc[1840338] = ['Director'] + ["Daniel Dahlstrom"]

movies with id: 1617643,1646521,1648186,1725370,1741482,1883229,1894772 will be deleted along with every reference

In [41]:
#multiple execution of this block could bring some errors
movies_df = movies_df.drop([1617643,1646521,1648186,1725370,1741482,1883229,1894772]).copy()
actors_df = actors_df.drop([1617643,1646521,1648186,1725370,1741482,1883229,1894772], errors='ignore').copy()
countries_df = countries_df.drop([1617643,1646521,1648186,1725370,1741482,1883229,1894772], errors='ignore').copy()
crew_df = crew_df.drop([1617643,1646521,1648186,1725370,1741482,1883229,1894772], errors='ignore').copy()
genres_df = genres_df.drop([1617643,1646521,1648186,1725370,1741482,1883229,1894772], errors='ignore').copy()
languages_df = languages_df.drop([1617643,1646521,1648186,1725370,1741482,1883229,1894772], errors='ignore').copy()
posters_df = posters_df.drop([1617643,1646521,1648186,1725370,1741482,1883229,1894772], errors='ignore').copy()
releases_df = releases_df.drop([1617643,1646521,1648186,1725370,1741482,1883229,1894772], errors='ignore').copy()
studios_df = studios_df.drop([1617643,1646521,1648186,1725370,1741482,1883229,1894772], errors='ignore').copy()
themes_df = themes_df.drop([1617643,1646521,1648186,1725370,1741482,1883229,1894772], errors='ignore').copy()
print()

In [42]:
movies_df[movies_df['name'].isna()]

,name,date,tagline,description,minute,rating
id,,,,,,


in the dataset there are movies with same name, and description but different minute and rating. it's not safe to delete duplicate that are not equal in every column of the dataset because a movies may have been produced in different versions

In [43]:
movies_df.loc[movies_df.duplicated(subset=['name','date','description','minute','rating'])]

,name,date,tagline,description,minute,rating
id,,,,,,
1073819,Better Than Sex,2000,3 days in the life of a 1 night stand.,After a one-night stand a couple is faced with...,84.0,3.24
1073820,Amoklauf,1994,<NA>,A nameless waiter spends his days sitting arou...,65.0,NaN
1073822,Still Working 9 to 5,2022,<NA>,Examines the 40-year evolution of gender inequ...,95.0,NaN
1073823,Power On: The Story of Xbox,2021,Four gamers. One idea. And a billion dollar bet.,Xbox almost didn't happen. Find out why in thi...,NaN,3.61
1073824,Crossing the Line,2006,<NA>,"In 1962, a U.S. soldier sent to guard the peac...",94.0,3.44
...,...,...,...,...,...,...
1940234,Medo,2023,<NA>,<NA>,4.0,NaN
1940448,Riina - Le verità nascoste,2018,<NA>,<NA>,45.0,NaN
1940456,Yen Ching 燕京,2018,<NA>,An intergenerational dilemma between immigrant...,64.0,NaN


removing the duplicates

In [44]:
movies_df = movies_df.loc[~movies_df.duplicated(subset=['name','date','description','minute','rating'])].copy()
number_of_duplicate = movies_df.duplicated(subset=['name','date','description','minute','rating']).sum()
print("Number of duplicates:" ,number_of_duplicate)

Number of duplicates: 0


### Actors

In [45]:
actors_df

,name,role
id,,
1000001,Margot Robbie,Barbie
1000001,Ryan Gosling,Ken
1000001,America Ferrera,Gloria
1000001,Ariana Greenblatt,Sasha
1000001,Issa Rae,Barbie
...,...,...
1941596,Marc Ma,Ba Cai/巴莱
1941596,线雨轩,Tata/塔塔
1941596,Jiang Yixuan,Zuo Yila（Zoila）/佐伊拉


In [46]:
actors_df.loc[actors_df['name'].isna()]

,name,role
id,,
1443629,<NA>,<NA>
1469981,<NA>,Self
1474958,<NA>,Cinematography
1773264,<NA>,<NA>


there is no way to reach these names

In [47]:
actors_df = actors_df.loc[~actors_df['name'].isna()].copy()
actors_df

,name,role
id,,
1000001,Margot Robbie,Barbie
1000001,Ryan Gosling,Ken
1000001,America Ferrera,Gloria
1000001,Ariana Greenblatt,Sasha
1000001,Issa Rae,Barbie
...,...,...
1941596,Marc Ma,Ba Cai/巴莱
1941596,线雨轩,Tata/塔塔
1941596,Jiang Yixuan,Zuo Yila（Zoila）/佐伊拉


In [48]:
actors_df.duplicated(subset=['name']).count()

np.int64(5798445)

In [49]:
countries_df

,country
id,
1000001,UK
1000001,USA
1000002,South Korea
1000003,USA
1000004,Germany
...,...
1941593,China
1941594,USA
1941595,USA


In [50]:
countries_df.loc[countries_df['country'].isna()]

,country
id,


countries have a lot of duplicates because each movie could have been  recorded in one or more countries, there is no need to delete duplicates is this case

In [51]:
crew_df

,role,name
id,,
1000001,Director,Greta Gerwig
1000001,Producer,Tom Ackerley
1000001,Producer,Margot Robbie
1000001,Producer,Robbie Brenner
1000001,Producer,David Heyman
...,...,...
1941596,Editor,Eric Kwong Chi-Leung
1941596,Cinematography,Kenny Tse
1941596,Composer,胡小欧


In [52]:
crew_df.loc[crew_df['name'].isna()]

,role,name
id,,
1859397,Writer,<NA>


In [53]:
crew_df = crew_df[~crew_df['name'].isna()].copy()

dataframe crew have a lot of duplicates because each crew member participated in multiple movies

In [54]:
crew_df.loc[crew_df.duplicated()]

,role,name
id,,
1000005,Camera operator,Ari Robbins
1000005,Stunts,Danny Downey
1000005,Sound,Ai-Ling Lee
1000005,Sound,Dan O'Connell
1000005,Hairstyling,Fríða Aradóttir
...,...,...
1941596,Casting,线雨轩
1941596,Editor,Eric Kwong Chi-Leung
1941596,Cinematography,Kenny Tse


In [55]:
genres_df

,genre
id,
1000001,Comedy
1000001,Adventure
1000002,Comedy
1000002,Thriller
1000002,Drama
...,...
1941563,Drama
1941566,Crime
1941569,Crime


In [56]:
genres_df.loc[genres_df['genre'].isna()]

,genre
id,


In [57]:
genres_df.loc[genres_df.duplicated()]

,genre
id,
1000002,Comedy
1000003,Adventure
1000003,Comedy
1000004,Drama
1000005,Drama
...,...
1941563,Drama
1941566,Crime
1941569,Crime


In [58]:
languages_df

,type,language
id,,
1000001,Language,English
1000002,Primary language,Korean
1000002,Spoken language,English
1000002,Spoken language,German
1000002,Spoken language,Korean
...,...,...
1941593,Language,Chinese
1941594,Language,English
1941595,Language,English


In [59]:
languages_df.loc[languages_df[['type','language']].isna().all(axis=1)]

,type,language
id,,


In [60]:
posters_df

,link
id,
1000001,https://a.ltrbxd.com/resized/film-poster/2/7/7...
1000002,https://a.ltrbxd.com/resized/film-poster/4/2/6...
1000003,https://a.ltrbxd.com/resized/film-poster/4/7/4...
1000004,https://a.ltrbxd.com/resized/film-poster/5/1/5...
1000005,https://a.ltrbxd.com/resized/film-poster/2/4/0...
...,...
1941593,<NA>
1941594,<NA>
1941595,https://a.ltrbxd.com/resized/film-poster/1/1/8...


In [61]:
posters_df.loc[posters_df['link'].isna()]

,link
id,
1014019,<NA>
1015381,<NA>
1016468,<NA>
1018453,<NA>
1027194,<NA>
...,...
1941590,<NA>
1941591,<NA>
1941593,<NA>


in this case removing the <NA> values will keep the data a bit cleaner


In [62]:
posters_df = posters_df[~posters_df['link'].isna()].copy()

In [63]:
posters_df.loc[posters_df.duplicated()]

,link
id,
1073819,https://a.ltrbxd.com/resized/film-poster/3/4/5...
1073820,https://a.ltrbxd.com/resized/film-poster/3/1/0...
1073822,https://a.ltrbxd.com/resized/film-poster/7/6/0...
1073823,https://a.ltrbxd.com/resized/film-poster/8/2/2...
1073824,https://a.ltrbxd.com/resized/film-poster/3/7/4...
...,...
1829507,https://a.ltrbxd.com/resized/film-poster/1/1/4...
1864503,https://a.ltrbxd.com/resized/film-poster/1/1/8...
1897229,https://a.ltrbxd.com/resized/film-poster/1/0/1...


removing duplicated posters

In [64]:
posters_df = posters_df[~posters_df['link'].duplicated()].copy()

In [65]:
releases_df

,country,date,type,rating
id,,,,
1000001,Andorra,2023-07-21,Theatrical,<NA>
1000001,Argentina,2023-07-20,Theatrical,ATP
1000001,Australia,2023-07-19,Theatrical,PG
1000001,Australia,2023-10-01,Digital,PG
1000001,Austria,2023-07-20,Theatrical,<NA>
...,...,...,...,...
1940967,USA,1909-01-01,Theatrical,<NA>
1940968,Sweden,1908-11-11,Theatrical,<NA>
1940969,France,1902-01-01,Theatrical,<NA>


In [66]:
releases_df.loc[releases_df[['country','date','type','rating']].isna().all(axis=1)]

,country,date,type,rating
id,,,,


checking duplicates in each movie, adding index as helper column

In [67]:
releases_df['index'] = releases_df.index
releases_df.loc[releases_df.duplicated(subset=['index','country','date','type'])]

,country,date,type,rating,index
id,,,,,


removing index helper column

In [68]:
releases_df.drop(['index'], axis=1)

,country,date,type,rating
id,,,,
1000001,Andorra,2023-07-21,Theatrical,<NA>
1000001,Argentina,2023-07-20,Theatrical,ATP
1000001,Australia,2023-07-19,Theatrical,PG
1000001,Australia,2023-10-01,Digital,PG
1000001,Austria,2023-07-20,Theatrical,<NA>
...,...,...,...,...
1940967,USA,1909-01-01,Theatrical,<NA>
1940968,Sweden,1908-11-11,Theatrical,<NA>
1940969,France,1902-01-01,Theatrical,<NA>


In [69]:
studios_df

,studio
id,
1000001,LuckyChap Entertainment
1000001,Heyday Films
1000001,NB/GG Pictures
1000001,Mattel
1000001,Warner Bros. Pictures
...,...
1941596,上海猫眼影业有限公司
1941596,坏小子（北京）传媒有限公司
1941596,亚太国影（重庆）文化传媒有限公司


In [70]:
studios_df.loc[studios_df['studio'].isna()]

,studio
id,
1259717,<NA>
1350206,<NA>
1450762,<NA>
1565428,<NA>
1579904,<NA>
1590891,<NA>
1605792,<NA>
1648488,<NA>
1706513,<NA>


in this case removing the <NA> values will keep the data a bit cleaner

In [71]:
studios_df = studios_df[~studios_df['studio'].isna()].copy()

In [72]:
studios_df['index'] = studios_df.index
studios_df.loc[studios_df.duplicated(subset=['index','studio'])]

,studio,index
id,,
1000044,Working Title Films,1000044
1000165,Working Title Films,1000165
1000263,Working Title Films,1000263
1000514,Fabula,1000514
1000547,Working Title Films,1000547
...,...,...
1852028,ARTE,1852028
1859604,ARTE,1859604
1863229,Star Media,1863229


it is possible to see that a single studio can appear multiple time in the dataframe for each movie. This could be it because a studio could have worked in different areas of the movie filming session but the dataframe does not show all the different area so the duplicates will be removed.

In [73]:
studios_df = studios_df[~studios_df['studio'].duplicated()].copy()

removing the helper column


In [74]:
studios_df.drop(['index'], axis=1)

,studio
id,
1000001,LuckyChap Entertainment
1000001,Heyday Films
1000001,NB/GG Pictures
1000001,Mattel
1000001,Warner Bros. Pictures
...,...
1941514,TCGT Entertainment
1941522,Trailblazing Films & Entertainment
1941534,Lo Vi Films


In [75]:
themes_df

,theme
id,
1000001,Humanity and the world around us
1000001,Crude humor and satire
1000001,Moving relationship stories
1000001,Emotional and captivating fantasy storytelling
1000001,Surreal and thought-provoking visions of life ...
...,...
1835643,Noir and dark crime dramas
1835643,Intriguing and suspenseful murder mysteries
1849827,Faith and religion


In [76]:
themes_df.loc[themes_df['theme'].isna()]

,theme
id,


In [77]:
themes_df['index'] = themes_df.index
themes_df.loc[themes_df.duplicated(subset=['index','theme'])]

,theme,index
id,,


In [78]:
dataloader._save_data()